# Intermediate Neural Network in Keras

Bulding an Intermediate Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

#### Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [ ]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [ ]:
X_train /= 255
X_test /= 255

In [ ]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Design Neural Network architecture

In [ ]:
model = Sequential()
# Dense = Fully Connected layer
model.add(Dense((64), activation = 'relu', input_shape = (784,)))
model.add(Dense((10), activation = 'softmax'))

In [ ]:
model.summary()

#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = SGD(lr = 0.1), metrics = ['accuracy'])

#### Train!

In [ ]:
model.fit(X_train, y_train, batch_size = 128, epochs = 20, verbose = 1, validation_data = (X_test, y_test))

#### Test Prediction 

In [ ]:
import scipy
from PIL import Image
from scipy import ndimage

import matplotlib.pyplot as plt
%matplotlib inline

test_image = "mnist_image_8.jpg"

# Preprocess image to fit the algorithm.
fname = "images/" + test_image
image = np.array(ndimage.imread(fname, flatten = False))
test_image = image.reshape((1, 28 * 28 * 1))
image_prediction = model.predict_classes(test_image)

plt.imshow(image)
print("The algorithm predicts: y = " + str(np.squeeze(image_prediction)))